# Lab Recap

Can we retrieve 3 random facts from the [Internet Chuck Norris Database](http://www.icndb.com/api/) and display them as formatted HTML?

# Sorting

Sorting algorithms are one of the most frequently studied class of algorithms. Not only are there many such algorithms, it is also very easy to implement them incorrectly.

Can we implement [Bubble sort](https://en.wikipedia.org/wiki/Bubble_sort) in Python? How can we test it?

See `sorting.py`.

Can we also implement [merge sort](https://en.wikipedia.org/wiki/Merge_sort)?

See `sorting.py`.

# Error Handling

What do we do when things go wrong?

## Using `None`

Consider a function `find(needle, haystack)` that finds the position of `needle` in the list `haystack`. Can we implement this function?

Does python have built-in functionality for doing this?

## Exceptions

Can we change our `find()` function so that it raises an exception?

Consider the function, `score(card_points, cards)`, from the week 4 lab. It takes in a dictionary representing point values for different cards and a list of cards, and calculates the total score for that list.

Can we implement this function differently using exceptions?

A number, `x`, inside a list, `list`, is said to lead back to itself in 1 step if `list[x] == x`. A number leads back to itself in 2 steps if `list[list[x]] == x`. Which of the numbers in the following list lead back to themselves in 3 steps.

In [ ]:
list = [0, 2, 1, 4, 5, 3, 9, 6, 7]

# Alternative ways of working with data structures

Can we implement the `score(card_points, cards)` function without conditionals or exception handling?

## List/Dictionary Comprehensions

Can we extract a list of zids from these email addresses?

In [ ]:
emails = ["z1234567@student.unsw.edu.au", "z7654321@unsw.edu.au", "z7891234@ad.unsw.edu.au", "z1357924@student.unsw.edu.au"]

Can we create a dictionary with the zids as keys and the email addresses as values?

Find all the words that are palindromes in this list:

In [ ]:
words = ["kayak", "hello", "racecar", "madam", "moon", "noon", "shish", "level"]